In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rasterio joblib numpy

In [ ]:
import rasterio
from rasterio import windows

def get_split_windows(width, height, splits=4):
    """Generate windows to split the raster into equal parts."""
    w, h = width // splits, height // splits
    for i in range(splits):
        for j in range(splits):
            yield windows.Window(col_off=i*w, row_off=j*h, width=w, height=h)

def save_window(raster_path, window, idx, output_folder):
    """Save a specific window of the raster as a new file."""
    with rasterio.open(raster_path) as src:
        window_transform = src.window_transform(window)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": window.height,
            "width": window.width,
            "transform": window_transform
        })

        output_path = f"{output_folder}/part_{idx + 1}.tif"

        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(src.read(window=window, out_shape=(src.count, window.height, window.width)))

# Update paths !!!
raster_path = '/content/drive/My Drive/xxx.tif'
output_folder = '/content/drive/My Drive/xxx/split_rasters'

with rasterio.open(raster_path) as src:
    width, height = src.width, src.height
    split_windows = list(get_split_windows(width, height, splits=4))

for idx, window in enumerate(split_windows):
    save_window(raster_path, window, idx, output_folder)


In [ ]:
from joblib import load
import rasterio
import numpy as np

model_path = '/content/drive/My Drive/xxx/XGBoost_best_model.joblib'
part_raster_path = '/content/drive/My Drive/xxx/split_rasters/part_16.tif'
output_prediction_path = '/content/drive/My Drive/xxx.tif'


model = load(model_path)


with rasterio.open(part_raster_path) as src:
    meta = src.meta.copy()
    data = src.read()

    predictions = model.predict(data.reshape(src.count, -1).T).reshape(src.height, src.width)


    meta.update(count=1, dtype=predictions.dtype)

    with rasterio.open(output_prediction_path, 'w', **meta) as dst:
        dst.write(predictions, 1)
